In [ ]:
import pandas as pd

import config

In [ ]:
from framework.data.data_loader import DataLoader

df = None

data_loader = DataLoader()
df = data_loader.load_data_from_disk(symbol=config.SYMBOL, timeframe=config.TIMEFRAME, suffix="_raw")

if df is not None:
    print(f"Data Rows: {len(df)}, Columns: {', '.join(df.columns)}")

In [ ]:
from framework.analysis.technical_indicators import TechnicalIndicators

technical_indicators = TechnicalIndicators()

if df is None:
    df = data_loader.fetch_historical_data(symbol=config.SYMBOL, timeframe=config.TIMEFRAME, days=365)

    df = technical_indicators.add_indicators(df)

    data_loader.save_to_csv(df, symbol=config.SYMBOL, timeframe=config.TIMEFRAME, suffix="_raw")

    print(f"Data Rows: {len(df)}, Columns: {', '.join(df.columns)}")

In [ ]:
from sklearn.model_selection import train_test_split

# First split: 70% Train, 30% Temp (Val + Test)
train_df, temp_df = train_test_split(df, test_size=0.3, shuffle=False)

# Second split: Split the 30% Temp into 50% Val (15% total) and 50% Test (15% total)
val_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=False)

# Reset indices to ensure environment works correctly
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
from framework.analysis.technical_indicators import TechnicalIndicators

technical_indicators.fit_scalers(train_df)

train_df = technical_indicators.normalize(train_df)
val_df = technical_indicators.normalize(val_df)
test_df = technical_indicators.normalize(test_df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

plot_df = pd.concat([train_df, val_df, test_df], axis=0)

# Use 4 columns for a balanced 8x4 Grid
cols = 4
rows = math.ceil(len(plot_df.columns) / cols)

plt.figure(figsize=(20, 5 * rows))  # Adjust figsize for clarity
sns.set_style("whitegrid")

for i, column in enumerate(plot_df.columns):
    plt.subplot(rows, cols, i + 1)

    # Plot histogram with KDE
    sns.histplot(plot_df[column], kde=True, bins=50, color="royalblue", edgecolor="black", linewidth=0.5)

    plt.title(f"Distribution: {column}", fontsize=12, fontweight="bold")
    plt.xlabel("Normalized Value")
    plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
features = [col for col in train_df.columns if col not in ["open", "high", "low", "close", "volume", "atr"]]

len(features), features

In [ ]:
# Compute correlation matrix on feature columns only
corr_matrix = plot_df[features].corr(method="pearson")

# Save to CSV
corr_matrix.to_csv("framework/analysis/pearson_correlation.csv")
print(f"Saved correlation matrix: {corr_matrix.shape}")

fig, ax = plt.subplots(figsize=(20, 16))
sns.heatmap(
    corr_matrix,
    ax=ax,
    annot=True,           # Show numbers in each cell
    fmt=".2f",            # 2 decimal places
    cmap="RdYlGn",        # Red = negative, Yellow = neutral, Green = positive
    center=0,             # Anchor color scale at 0
    vmin=-1, vmax=1,
    linewidths=0.5,
    annot_kws={"size": 7},
    square=True,
)
ax.set_title("Feature Correlation Heatmap", fontsize=16, fontweight="bold", pad=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right", fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=8)
plt.tight_layout()
plt.show()

# Extract pairs with |correlation| > threshold (excluding self-correlation)
THRESHOLD = 0.85

high_corr_pairs = (
    corr_matrix
    .where(lambda df: (df.abs() > THRESHOLD) & (df != 1.0))
    .stack()
    .reset_index()
    .rename(columns={"level_0": "Feature A", "level_1": "Feature B", 0: "Correlation"})
    .drop_duplicates(subset=["Correlation"])   # Remove duplicates (A-B and B-A)
    .sort_values("Correlation", key=abs, ascending=False)
)

print(f"Found {len(high_corr_pairs)} highly correlated pairs (|corr| > {THRESHOLD}):")
high_corr_pairs

In [ ]:
from framework.ai.training.train_agent import train

train(train_df, features)

In [ ]:
from framework.ai.training.train_agent import evaluate

evaluate(val_df, features, "Validation")

In [ ]:
evaluate(test_df, features, "Test")